**task1 - summarization**

In [ ]:
!pip install transformers peft

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.6/251.6 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 26.2 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-many

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
from peft import PeftModel, PeftConfig

# Load the base model and tokenizer
model_name = "csebuetnlp/mT5_multilingual_XLSum"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Verify and correct the path to the LoRA adapters
adapter_path = "/content/drive/MyDrive/fine-tuning-project/task1/fine-tuned-task1/"  # Double-check this path
peft_config = PeftConfig.from_pretrained(adapter_path)

# Load the LoRA adapters from "adaptor_model" directory
model = PeftModel.from_pretrained(model, adapter_path,  load_in_8bit=True, device_map='auto')

/usr/local/lib/python3.10/dist-packages/transformers/convert_slow_tokenizer.py:560: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [ ]:
# Define a sample input
sample_article = """به گزارش روابط عمومی صباایده، این برند فعال در حوزه ویدئو، استریم و تبلیغات آنلاین که پلتفرمهایی چون فیلیمو، آپارات و آژانس تبلیغات آنلاین صباویژن را زیر مجموعه خود دارد، با سرمایه گذاری در یکی از دو استارتاپ بزرگ حوزه گردشگری، به طور رسمی وارد بازار فروش و رزرواسیون بلیط هواپیما و قطار شد.
 ظرفیت‌های موجود در بازار گردشگری آنلاین ایران و فاصله زیادی که این حوزه از بازار آفلاین دارد، رشد قابل‌توجه و ارگانیک فلایتیو از ابتدای فعالیت آن و ویژگی‌های مشترک تیم‌های مدیریتی دو برند که آن‌ها را بهم نزدیک‌تر می‌کند از مهمترین علت‌های جدیدترین سرمایه‌گذاری صباایده است.
 این سرمایه‌گذاری که با خرید ۲۰ درصد سهام فلایتیو رقم خورده پیرو تجربه و دانش فلایتیو در صنعت گردشگری و در کنار ظرفیت‌ها و حمایت‌های گسترده تبلیغاتی صباایده، مسیر تازه‌ای را در صنعت گردشگری آنلاین ایران رقم خواهد زد.
"""

# Preprocess the input
input_ids = tokenizer(
    [sample_article],
    return_tensors="pt",
    padding="max_length",
    truncation=True,
    max_length=512
)["input_ids"]

# Generate the summary
output_ids = model.generate(
    input_ids=input_ids,
    max_length=84,
    no_repeat_ngram_size=2,
    num_beams=4
)[0]

# Decode the summary
summary = tokenizer.decode(
    output_ids,
    skip_special_tokens=True,
    clean_up_tokenization_spaces=False
)

print(summary)

صباایده، یکی از بزرگترین شرکت های گردشگری آنلاین ایران، با سرمایه گذاری بیش از ۲۰ درصد سهام خود در بازار فروش و رزرواسیون بلیط هواپیما و قطار وارد بازار شد.


**task2 - sentiment analysis**

In [ ]:
from transformers import BertTokenizer,BertForSequenceClassification
from peft import PeftModel, PeftConfig

model = BertForSequenceClassification.from_pretrained('HooshvareLab/bert-base-parsbert-uncased', num_labels=2)
tokenizer = BertTokenizer.from_pretrained('HooshvareLab/bert-base-parsbert-uncased')

# Verify and correct the path to the LoRA adapters
adapter_path = "/content/drive/MyDrive/fine-tuning-project/task2"  # Double-check this path
peft_config = PeftConfig.from_pretrained(adapter_path)

model = PeftModel.from_pretrained(model, adapter_path,  load_in_8bit=True, device_map='auto')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at HooshvareLab/bert-base-parsbert-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
import torch

# Sample input text
sample_text = "جوجه کباب خیلی پخته و خشک شده بود"

# Tokenize the sample text
inputs = tokenizer(sample_text, return_tensors="pt", padding=True, truncation=True, max_length=128)

# Perform inference
with torch.no_grad():
    outputs = model(**inputs)
    predictions = torch.argmax(outputs.logits, dim=-1)

# Map the prediction to the sentiment label
label_map = {0: "negative", 1: "positive"}
predicted_label = label_map[predictions.item()]

print(f"Input Text: {sample_text}")
print(f"Predicted Sentiment: {predicted_label}")


Input Text: جوجه کباب خیلی پخته و خشک شده بود
Predicted Sentiment: negative


این هندس فری تازه به دستم رسیده و برای کسایی که انتظار شنیدن یک موسیقی با کیفیت دارن حتما پیشنهاد میکنم بعد از شنیدن آهنگ با این هدست آدم به این فکر میکنه انگار تا حالا آهنگ نشنیده
1
عالی بود عالی، امیدوارم کیفیت همینجور بمونه
1
من وقتی این ایرباد به لپ تاپ ایسوس وصل میکنم و با نرم افزار حرفه ای آهنگها را پخش میکنم صدای گرم و شفافی بهم میده و نهایت قدرت این ایرباد نشون میده
1
با عرض سلام ضمن عرض خسته نباشید چلوکباب کوبیده سفارشی بسیار عالی بود ولی جوجه در چلو جوجه کباب مخصوص قدری خشک بود در مجموع از نحوه سرویس دهی و زمان بندی و کیفیت غذا رضایت کامل داشته و تشکر فراوان دارم
1
خوشمزه و داغ مخصوصا با سس چیلی که فرستاده بودن بسیار خوب بود ممنون ازجوجه داغ و اسنپ
1
متاسفانه با وجود تاکید زیاد به خشک و برشته نبودن نان‌ها، نان سنگک به قدری خشک و بیسکویتی بود که قسمت عمده آن در بسته بندی کاملا خرد شده بود اطراف نان کاملا سوخته بود واقعا ناراضی بودم و خرید مجدد نخواهم کرد
0
طعم چیزکیک اوکی بود عالی نبود ولی اوکی بود اما بسته‌بندی و ارسالش بسیار نامناسب بود کیک کاملا از ریخت افتاده و به بدنه‌های جعبه مالیده بود
0
جوجه کباب خیلی پخته و خشک شده بود
0
سالاد فصل بسیار کیفیت بدی داشت وکاهوداخل ساندویچ از کیفیت خوبی برخوردارنبود
0
من تا مایع ظرفشویی سفارش دادم ولی یکی ارسال شد
0
فقط‌میتونم بگم واقعا متاسفم واسه یه رستورانی که واقعا سال پیش فوق العاده بود و الان انقدر افت کرده سیب زمینی بسته پر شده بود و پیتزا خانوادش حتی اندازه ی‌ پیتزا نفره جاهای دیگه مخلفات نداشت واقعا متاسفم
0
عالی بود زمان بندی بسته بندی کیفیت و کمیت مخصوصا دورچین غذا واقعا عالی بود
1
کیفیت اصلا خوب نبود، خمیر بود، دیگه نمیگیرم
0
این کیک نفره هم نیست ولی روش نوشته تا نفر ابروم رفت واقعا اطلاعات غلط نذارین
0
هربار سفارش سرد می‌رسه با اینکه در توضیحات سفارش تاکید کردم، در ضمن به جای نوشابه، اسپرایت فرستادند،
0
پیتزا سرد بود و گوشتهای داخلش نامرغوب
0
کباب کوبیده معمولی بود، اما ماهی قزل آلا بی مزه بود و بسیار بوی زخم میداد
0
با دلیل کیفیت پایین بسته بندی و داغ بودن غذا جعبه مقوایی به طور کامل بر روی پیتزا چسبیده بود به طوری که مجبور به دور ریختن قسمت عمده غذا شدم
0
سفارش به قدری دیر رسید که غذا کاملا سرد شده بود سیبزمینی از سردی بیات شده بود پیتزا و ساندویچ کاملا سرد شده بودند
0
سلام سالاد خوشمزه‌ای بود و ترکیباتش مناسب بود ولی مقدار فیله نسبتا کم بود و کاهوهاش بزرگ بودند بهتره بیشتر ریز شوند با تشکر
1
غذا خوب، تازه، گرم و خوشمزه بود با تشکر از مدیریت رستوران مهدی و همچنین تشکر ویژه از اسنپ فود
1
کاملا روی چیز کیک‌ها سوخته بود و به خاطر این مشکل طعمش هم خیلی بد شده بود
0
زیاد تعریفی نداشت خیلی حیلی معمولی بود
0
با کلی تاخییر، اشتباه هم تحویل دادند و عدم پشتیبانی مناسب اسنپ فود
0
رفتار پیک مودبانه بود
1
غذا کاملا سرد قارچ‌ها مزه ابلیمو میداد متاسفانه
0
پاستاش خیلی تند بود و سسش خوشمزه نبود
0
سس سیب زمینی خیلی سر و سفت بود و نمیشد ریخت روی سیب زمینی
1
هم سریع و هم داغ واقعا ممنونم
1
لطفا به سرآشپزتون بگین اینقدر گل و گیاه نزنه به غذا …نمیفهمم چرا باید به فیله استریپس آویشن بزنن …یا به سیب زمینی برگبو …کیفیت خوب بود ولی طعمشو خراب کرده بود
0
برش شیرینی رولت خیلی نازک بود اما هر شیرینی خوری وقتی مزه قصر زیر زبونش طنین انداز بشه جادوی شیرینی پز رو خیلی خوب حس میکنه
1
هرچی تعریف کنم ازتون کمه طعم پیتزاتون فوق العاده بود خوشمزه‌ترین پیتزای عمرم بود دمتون گرم
1
بادام هندی خرد شده بود، بدترین نوع تخمه کدو گوشتی تحویل من دادن و پسته هم به نسبت قیمت اصلا ارزش خرید نداره
0
اون چیزی که سفارش داده میشه نمیاد هرچیزی داشته باشند رو میارن
0
یکم شیرین‌تر از قبل بود که به سلیقه من خوب نبود
1
کاملا بی کیفیت بود بدلیل ذیغ وقت دریافت کردم
0
جای شیر پرچرب کم چرب ارسال شده بود
0
در نحوه تحویل سفارش عدم حضور پیک و گذاشتن غذا در آسانسور بدون اطلاع
0
آب هویج بوی خاک میداد آب سیب سیاه شده بود آب آناناس هم یک چهارمش خالی بود آب پرتغال هم تلخ تلخ بود اصلا خوب نبود کاملا ناراضی هستم
0
چیز کیک رزبری پنیرش زیاد بود، میزان کیکش خیلی کم بود جوریدن لازم داشت، به نظرم باید کیکش بیشتر باشه چیزش کمتر
1
قبل از ارسال تماس گرفتن از سوپر که بستنی مورد نظر موجود نیست پولش را همراه بقیه خرید میفرستن ولی نه پول را فرستادن نه به اعتبار اضافه شد
0
همیشه از اینجا خرید میکنم بهترین تو منطقه همیشه عالی بوده خوشحالم که راحت سفارش میدم و در ضمن کیفیت شیرینی عالی بود و مواد اولیه عالی
1
حدود ساعت طول کشید
0
سفارش دادم هنوز به دستم نرسیده با توجه به نقد بد نیست البته اگه دیجی کالا ارسال ش کنه نقد هم میزارم
1
باسلام و سپاس از عوامل اسنپ فوت درود برشما
1
اصلا شبیه عکس نبود و واقعا نا مناسب بود پنکیک بدون هیچ طعم دهنده میوه‌های بی کیفیت و اینکه همه‌ی خوراکی‌ها حتی نیمرو قاطی باهم در یک ظرف بودند افتضاح
0
بوی ضخم گوشت خیلی بد بود و نمیشد خورد
0
کیک موکا اسلایسی اصلا تازه نبود مشخص بود که حداقل دو روز توی یخچال بوده از ناتالی انتظار نداشتیم
0
با سلام حجم ساندویچ کاملا کم و نصف نان باگت بود من که اصلا سیر نشدم حداقل ساندویچ رو از وسط نصف نکنید از نظر روانی یکم آدم راحت باشه، قبلا خیلی خوب بود در کل اصلا راضی نبودم
0
حجم خوب بود فقط یک سوال؟ مرغ بریان که روی آتیش درست میشه چجوری اونهمه روغن تو ظرف بود؟ خوب بود راضی هستم
1
مخلفات تازه و عالی خود ابگوشت هم کمی بی مزه بود ولی به نسبت قیمت با تخفیفش خوب بود مرسی
1
عالی بود تازه و داغ خرد و بسته بندی شده
1
عالی بود مثل همیشه ساندویچش رو برای اولین بار تست میکردم چقدر خوشمزه است عاشق ترکیب و نونش شدم
1
گوشتی که ماندیدیم بیشتر میخورد سبزیجات باشه تا گوشتو قارچ
0
مثل همیشه به توضیحات توجه کرده بودید و بسیار عالی بود خیلی ممنون از شما و‌پیک‌محترم تشکر
1
بهتره بجای حجم بالای سفیده تخم مرغ بجای خامه از خامه استفاده شود
0
سلام دوستان من این گوشی نسبت به آیفون الجی وی الجی جی مقایسه میکنم هر سه این گوشی ها رو دارم قدرت پردازنده مرکزیشون و رم داخلی گوشی های الجی رو کاغذ هر سه یکسان و خیلی بالاتر از آیفون هست ولی با این حال آیفون تو بازی ها خیلی بهتر و سرتره نکته بعدی با اینکه پلاس خیلی بعد از معرفی شده ولی کیفیت دوربین سلفیش به شدت از و دوربین پشتش از پایین تره حالا چرا من این گوشی گرفتم گیگابایت حافظه داخلی با یه مموری گیگ با یه اکانت اسپاتیفای گزینه روشن هدفن بسیار عالی ساعت ها با بهترین کیفیت صدا موزیک گوش بدید صفحه نمایشش و همچنین قاب پشتش هم کاملا مورد جذبتون میکنه نکته مثبت دیگه کاهش بسیار زیاد قیمت گوشی توسط خوده الجی هست که فکر میکنم الان بعد حدود یکسال از عرضه گوشی می ارزه اگر قاب خواستید بخرید میتونید از قاب های هم استفاده کنید ولی برای گلس یا ضد خش صفحه نمایش باید حتما برای تهیه کنید
1
با توجه به مسافت هزینه ارسال زیاد بود و خیلی دیر رسید
1
به نظرم طبخش میتونست خیلی خیلی مطلوب‌تر از این باشه …
0
خسته نباشید میگم خدمتتون، نسبت به قیمت بسیار عالی بود امیدوارم که روز به روز بهتر از قبل باشید لطفا کیفیت را خراب نکید
1
تاریخ روز‌اجناس را بفرستند خوشحال میشیم
1
من هر بار که از این رستوران غذا سفارش میدم بیشتر از قبل عاشقش میشم کیفیت عالی حجم خوب و به اندازه، بسته بندی خوب قیمت خوب واقعا دست مریزاد و ممنونم از رستوران باگت و آشپزش که انقدر خارق العادس مرسی مرسی
1
خیلی بد بوو دفعه پیش هم تازه بود هم زیاد اینبار خشک بود شیره نداشت و خیلی کمتر بود فک کنم نیم کیلویی دادید اشتباهی اینبار
0
نان سفارش دادیم بسیار سفت و سرد
0
خوب بود فقط یک مقدار سیب زمینی‌ها از تازگی افتاده بود
1
خسته نباشید مرغ عالی بود کاملا داغ رسید فقط مقدار سیب زمینی خیلی کمتر بود
1
خیلی سفت بود و به بستنی بیشتر نزدیک بود
1
عذم ارسال کوپن پوره سیب زمینی توسط گاردن گریل
0
سفارش بعد از دقیقه تازه ثبت شد و بعد از ٫ ساعت تحویل داده شد
0
خوب بود ولی زیادی شیرین بود موز و گردو هم خیلی کم بود
1
خیلی خوب بود وممنون بابت مخلفات
1
سرعتشون فوق العاده بود، کلا پنج دیقه نشد، کیفیت هم عالی بود …
1
فوق‌العاده لذیذ و سالم متشکر
1
عالی بود خسته نباشید
1
این سوپر مارکت جزو ارسال سریع‌ها بود و بسیار هم نزدیک هستند، ‌ سفارشها بعد دقیقه تحویل داده شد
0
قاشق و چنگال نذاشتید با چی باید غذا رو خورد با دست
1
پیتزا عالی سیب زمینی پنیرش خوب نبود سیب زمینی ساده‌اش هم روش فلفل سیاه ریخته بودند کاش نمی‌ریختن
1
شور بودن سس تریاکی ساندویچ
0
بسیار خوشمزه بود مثل همیشه ممنون بخاطر سوپ و همه‌ی تازگی و خوشمزگی و تمیزی و نظم کاری شما سلامت و موفق باشید
1
ژل ضدعفونی کننده بس به قیمت هزار تومن رو هزار تومن فاکتور کردن و قیمت درج شده رو پاک کردن وقتی هم که پیگیری کردیم گفتن گرون خریدیم، همینیه که هست شما بگید اسم این کار چیه؟
0
بسیار بد طعم و مونده
0
به موقع و کامل رسید
1
ساندویچ مرغ با سس رمولادا مثل همیشه بسیار لذیذ و خوشمزه
1
خیلی زیاد بود خیلی خوشمزه بود خیلی عالی بود
1
یکی از خوشمزه‌ترین ساندویچایی که خوردم بوده
1
بسیار بسیار بی کیفیت، توی عکس سه تا کوکتل درشت هست، ولی خود ساندویچ بزرگ کلا دوتا کوکتل کوچک نصف شده دارد
0
افتضاح بود نون بیات محتویات کم نسبت به قیمت بالا همجوره افتضاح بود
0
نان‌ها هر سه عدد کاملا تلخ بودن که برام خیلی عجیب بود
0
فلفلش خیلی تند بود
1
واقعاً عالیه البته ما از دیجی کالا نخریدم ولی از نظر وضوح تصویر و صدا واقعا راضی کننده است اما اگه خواستین بخرین اینچ و زیاد فرق ندارن حدود چهار پنج ماه خریدم راضی ام الجی در زمینه نمایشگر ها رقیب نداره
1
در سفارش من یک بسته تخم مرغ تایی تلاونگ بود که از تا یک دانه کاملن پوچ، سه تا شکسته قدیمی یعنی تقصیر حمل نبود وجود داشت تاریخ تولید آن هم در شرف انقضا در یک هفته
0
سلام همه چی خوب بود ولی فقط یه مورد که با اصلاحش خیلی کیفیتتون میره بالاتر اینکه بجای کاغذ که میپیچید دور ساندویچ یه چیزه بهتری مث زر ورقی چیزی ک نچسبه به ساندویچ خیلی بهتر میشه چون کاغذ که میمونه رنگش در میاد در کل همه چی خوب بود
1
چیکن استریپس خشک، کهنه و کم حجم بود و نسبت به قیمت اصلا نمی‌ارزید گافیست کیفیت و کمیت رو با رستورانهای هم رده از نظر قیمتی در همین محدوده مقایسه کنید-
0
چرا آماده سازی و ارسال یک پیتزای پپرونی باید دقیقه طول بکشه ؟
0
ما پاستا سفارش میدیم شما سیب زمینی و قارچ میفرستین؟
0